In [1]:
from utils.data_management import load_data
import config as cfg

import tensorflow as tf
import numpy as np
import os

2021-07-12 21:53:07.002125: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


# Chargement des données

## Chargement des données

In [2]:
# On profite de la puissance de Python : Les dictionnaires
datasets_map = {
    'cifar': None,
    'mnist': None,
    'fashion': None
}

for name in datasets_map.keys():
    datasets_map[name] = load_data(name, cfg.DATA_DIRECTORY)

# Auto-encodeur

Nous commençons par un auto-encodeur en TensorFlow à l'aide d'une classe personnalisée.

In [ ]:
class ConvolutionalAE(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super(AutoEncoder, self).__init__(*args, **kwargs)
        pass

    def encoder(self, x):
        # TODO : Voir si on n'utiliserait pas un override de tf.keras.Layer
        pass

    def decoder(self, x):
        # TODO : Voir commentaire dans encoder
        pass

    def call(self, inputs):
        pass